#  🐍 Python and the Deadly Diamond of Death 💎

The "diamond problem" (sometimes referred to as the "Deadly Diamond of Death") is an ambiguity that arises when two classes B and C inherit from A, and class D inherits from both B and C.

If there is a method in A that B and C have overridden, and D does not override it, then which version of the method does D inherit: that of B, or that of C?
<img src="https://uspgamedev.org/media/06_encontro_bcc_2014/images/diamond_of_death.png" width=300 align="center">


### Simple diamond

Below is a cell which show a simple diamond as per the above diagram. A is the base class which inherits from object. B and C both inherit from A and D inherits from both B and C. Both B and C override the methods of A. This should all be pretty straightforward OOP. When you run the next cell you can see this.

I apologise for the somewhat drole naming of these objects but it is in fact clearer than naming them something more fun like "animal", "lion", "tiger", and "liger".

In [1]:
class A:
    @classmethod
    def m(self):
        print("m of A called")

class B(A):
    @classmethod
    def m(self):
        print("m of B called")
    
class C(A):
    @classmethod
    def m(self):
        print("m of C called")

class D(B,C):
    pass

A.m()
B.m()
C.m()

m of A called
m of B called
m of C called


What will D inherit though? It does not override the methods of its parent classes.

Before you run the next cell, stop and predict what you think will be printed. Will it be the method of A, B, C, or some combination that will be called?

In [2]:
D.m()

m of B called


As you can see, D inherits the method of B. This is because B is specified first in the inheritance list when D was defined. In the cell below we reverse the inheritance order and you can see that now D inherits the method of C.

In [3]:
class D(C,B):
    pass

D.m()

m of C called


### Half a diamond

So multiple inheritance and the deadly diamond of death seems to be pretty easily understood and avoided in Python, correct? Well, let's test that and how well you understand it. In the cell below we've removed the m method of B. So B should inherit the method of A and C should override the method of A. When you run the cell below you can see this.

In [4]:
class A:
    @classmethod
    def m(self):
        print("m of A called")

class B(A):
    pass
    
class C(A):
    @classmethod
    def m(self):
        print("m of C called")

class D(B,C):
    pass

A.m()
B.m()
C.m()

m of A called
m of A called
m of C called


What does this mean for D though?

Before you run the next cell have a stop and predict what the method m of D will produce. Will D inherit the method m from B which inherits it from A or will D inherit the method m from C?

In [5]:
D.m()

m of C called


Whether you said "A" or "C" you are correct! (Although "C" is preferred)

If you're working in Python2 then D will inherit from A.

If you're working in Python3 (as you should be) then D will inherit from C.

If you want to have the same inheritance behaviour in Python2 as Python3 then you should make your root object (A) inherit from the base class object.

### Super and MRO

Let's extend our example now so that each method defines its own m method. We can see that thanks to the Liskov Substitution Principle that an object of class D called by other class names will result in the other class names being used.

In [6]:
class A:
    def m(self):
        print("m of A called")

class B(A):
    def m(self):
        print("m of B called")
    
class C(A):
    def m(self):
        print("m of C called")

class D(B,C):
    def m(self):
        print("m of D called")

In [8]:
d = D()
D.m(d)
B.m(d)
C.m(d)
A.m(d)

m of D called
m of B called
m of C called
m of A called


What about if we want to have D's method m also call the methods of its parent classes?

In [10]:
class D(B,C):
    def m(self):
        print("m of D called")
        B.m(self)
        C.m(self)
        A.m(self)

In [11]:
d = D()
d.m()

m of D called
m of B called
m of C called
m of A called


Let's do this for all the classes now and see what this looks like

In [12]:
class A:
    @classmethod
    def m(self):
        print("m of A called")

class B(A):
    @classmethod
    def m(self):
        print("m of B called")
        A.m()
    
class C(A):
    @classmethod
    def m(self):
        print("m of C called")
        A.m()

class D(B,C):
    @classmethod
    def m(self):
        print("m of D called")
        B.m()
        C.m()

In [13]:
B.m()
print('---')
D.m()

m of B called
m of A called
---
m of D called
m of B called
m of A called
m of C called
m of A called


Uh oh! We can clearly see that this causes an issue. When we call B's m method it works as expected as B's m method now calls A's m method.

The trouble is that it means that when D's m method is called it calls A's m method twice as both B and C's m methods call A's m method.

Before reading further, have a think about how you might solve this problem. See if you can think of both a unpython and a pythonic way of solving the problem!

### Unpythonic solution

The unpythonic solution is as per below. You have two methods for each class. One that is private and one that is public. This means that you can now have each m method only call itself and each of it parents' m methods once!

In [14]:
class A:
    @classmethod
    def m(self):
        print("m of A called")

class B(A):
    @classmethod
    def _m(self):
        print("m of B called")
    @classmethod
    def m(self):
        self._m()
        A.m()
    
class C(A):
    @classmethod
    def _m(self):
        print("m of C called")
    @classmethod
    def m(self):
        self._m()
        A.m()

class D(B,C):
    @classmethod
    def m(self):
        print("m of D called")
        B._m()
        C._m()
        A.m()

In [ ]:
D.m()
print('---')
B.m()

... but it really isn't very pretty is it?

Before reading the pythonic solution below, have another think about how you might solve this.

### Pythonic solution

Here is a super solution! Python's super function!

The super function refers to the parent and is especially useful when using the \_\_init\_\_ method. It ensures that each method will only be called once and in the correct order.

In [15]:
class A:
    @classmethod
    def m(self):
        print("m of A called")

class B(A):
    @classmethod
    def m(self):
        print("m of B called")
        super().m()
    
class C(A):
    @classmethod
    def m(self):
        print("m of C called")
        super().m()

class D(B,C):
    @classmethod
    def m(self):
        print("m of D called")
        super().m()

In [16]:
D.m()
print('---')
B.m()

m of D called
m of B called
m of C called
m of A called
---
m of B called
m of A called


In [17]:
class A:
    def __init__(self):
        print("A.__init__")

class B(A):
    def __init__(self):
        print("B.__init__")
        super().__init__()
    
class C(A):
    def __init__(self):
        print("C.__init__")
        super().__init__()


class D(B,C):
    def __init__(self):
        print("D.__init__")
        super().__init__()

In [18]:
d = D()

D.__init__
B.__init__
C.__init__
A.__init__


### Easy order discovery

The only remaining question is "How do I find out what order Python will use?". Well, Python will assess the tree for the class structure with a depth-first approach and then work from left to right. This is why (as you see above) it would prefer to use the m method of B, then C, then A.

If you can't understand why your class inheritance is not working and want to see tha actual order of method resolution that python is using, well then there's a method for that too!

The mro method shows you the "method resolution order" of an object and the order in which it will override/inherit methods.

In [19]:
A.mro()

[__main__.A, object]

In [20]:
B.mro()

[__main__.B, __main__.A, object]

In [21]:
C.mro()

[__main__.C, __main__.A, object]

In [22]:
D.mro()

[__main__.D, __main__.B, __main__.C, __main__.A, object]